In [1]:
import requests
import yaml
import pandas as pd

creds = yaml.safe_load((open('config.yaml')))

league_id = creds['league_id']

import os


In [2]:
week_start = 1
week_end = 4


In [6]:
#for each week loop through

for week in range(week_start, week_end+1):

    url = f'https://api.sleeper.app/v1/league/{league_id}/matchups/{week}'

    r = requests.get(url)

    matchups = r.json()
    

    ##create a blank list
    team_stats_list = []
    
    #for each team loop through to get their points in a given week as a dictionary
    for team in matchups:

        new_row = {'week': week, 'team': team['roster_id'], 'points': team['points'], 'matchup': team['matchup_id']}
        team_stats_list.append(new_row)

        # convert list to a dataframe
        team_stats = pd.DataFrame(team_stats_list)


        #create a copy to bring in opponent data, rename the columns
        opp_stats = team_stats.copy()
        opp_stats = opp_stats.rename(columns = {'team':'opp', 'points': 'opp_points'})

        #join on 
        team_stats = team_stats.merge(opp_stats)

        #remove records where team = opp
        team_stats = team_stats[team_stats['team'] != team_stats['opp']]

        #just in case
        team_stats = team_stats.drop_duplicates()


        #put it rawdata folder
        team_stats.to_csv(f'raw_data/team_stats_week_{week}.csv', index=False)
            


In [4]:

#Boom now check your raw_data folder

In [7]:
team_stats

,week,team,points,matchup,opp,opp_points
1,4,1,92.50,1,9,86.88
2,4,9,86.88,1,1,92.50
5,4,2,129.02,6,10,116.90
6,4,10,116.90,6,2,129.02
9,4,3,69.88,3,12,164.90
10,4,12,164.90,3,3,69.88
13,4,4,146.90,4,6,108.78
14,4,6,108.78,4,4,146.90
17,4,5,112.54,5,8,130.32
18,4,8,130.32,5,5,112.54


In [8]:
#create a list of csv file names, we can pass this list of strings into a for loop
file_list = []
for root, dirs, files in os.walk("raw_data"):
    if root == 'raw_data':
        for filename in files:
            file_list.append(filename)

file_list

['team_stats_week_1.csv',
 'team_stats_week_2.csv',
 'team_stats_week_3.csv',
 'team_stats_week_4.csv']

In [9]:
# get only files named team_stats_
# in capstone 2 we'll have player_stats_ files
team_files = [file for file in file_list if 'team_stats_week_' in file]


In [10]:
#create a blank team stats df
team_stats = pd.DataFrame()


# loop through, read the file as a pandas dataframe and concat it to our team_stats data frame
for file in team_files:
    a = pd.read_csv('raw_data/'+file)
    team_stats = pd.concat([team_stats,a])


#drop duplicates, removes the extra header rows
team_stats = team_stats.drop_duplicates()


#save to prod_data, may need to create a prod_data folder
team_stats.to_csv("prod_data/fantasy_team_stats.csv", index=False)

